# Import Library

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 8, 5
from sklearn.metrics import mean_absolute_error

# Generate Sin(x) Dataset

In [2]:
# define input array with angles from 60deg to 300deg converted to radians
x = np.array([i*np.pi/180 for i in range(60,300,4)])
np.random.seed(100)  #Setting seed for reproducability
y = np.sin(x) + np.random.normal(0,0.15,len(x)) #加上雜訊

data = pd.DataFrame(np.column_stack([x,y]),columns=['x','y'])
plt.plot(data['x'],data['y'],'.')

# Generate New Features with higher power 
自行產生新的feature，x^2～x^15

In [3]:
for i in range(2,16):  #power of 1 is already there
    colname = 'x_%d'%i      #new var will be x_power
    data[colname] = data['x']**i
data.head()

x         y       x_2       x_3       x_4       x_5       x_6  \
0  1.047198  0.603561  1.096623  1.148381  1.202581  1.259340  1.318778   
1  1.117011  0.950196  1.247713  1.393709  1.556788  1.738948  1.942424   
2  1.186824  1.100139  1.408551  1.671702  1.984016  2.354677  2.794587   
3  1.256637  0.913191  1.579137  1.984402  2.493673  3.133642  3.937850   
4  1.326450  1.117494  1.759470  2.333850  3.095735  4.106339  5.446854   

        x_7       x_8        x_9       x_10       x_11       x_12       x_13  \
0  1.381021  1.446202   1.514459   1.585938   1.660790   1.739176   1.821260   
1  2.169709  2.423588   2.707173   3.023942   3.377775   3.773011   4.214494   
2  3.316683  3.936319   4.671717   5.544505   6.580351   7.809718   9.268760   
3  4.948448  6.218404   7.814277   9.819710  12.339811  15.506664  19.486248   
4  7.224981  9.583578  12.712139  16.862020  22.366630  29.668222  39.353420   

        x_14       x_15  
0   1.907219   1.997235  
1   4.707635   5.258479  
2  11.000386  13.055521  
3  24.487142  30.771450  
4  52.200353  69.241170

# 0. Function definition

In [4]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge

In [5]:
def example_regression(data, power, plot_dict, reg_type, alpha = 0):
    # define estimator object
    type_dict = {'Linear':LinearRegression(normalize=True),
                 'Lasso':Lasso(alpha = alpha, normalize=True),
                 'Ridge':Ridge(alpha = alpha, normalize=True)}
    
    # generate X in order
    X = ['x']
    if power >=2:
        X.extend(['x_%d'%i for i in range(2,power+1)])
    
    # fit the model
    if reg_type in type_dict:
        model = type_dict[reg_type]
    model.fit(data[X],data['y'])
    y_pred = model.predict(data[X])
    mae = mean_absolute_error(y_pred, y)
    # check if a plot is to be made for the entered power
    if power in plot_dict:
        plt.subplot(plot_dict[power])
        plt.tight_layout()
        plt.plot(data['x'],data['y'],'.')
        plt.plot(data['x'],y_pred)
        plt.title('Plot for power: %d'%power + '\n' + 'mae:%.2f'%mae)
        
    # return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([model.intercept_])
    ret.extend(model.coef_)
    return ret

In [6]:
# initialize a dataframe to store the results:
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['pow_%d'%i for i in range(1,16)]

perf_Linear = pd.DataFrame(index=ind, columns=col)

# define the powers for which a plot is required: {power:where}
plot_dict = {1:231,3:232,6:233,9:234,12:235,15:236}

# iterate through all powers and assimilate results
for i in range(1,16):
    perf_Linear.iloc[i-1,0:i+2] = example_regression(data, power=i, plot_dict=plot_dict,reg_type='Linear')

In [7]:
pd.options.display.float_format = '{:,.2g}'.format
perf_Linear

rss intercept coef_x_1 coef_x_2 coef_x_3 coef_x_4 coef_x_5 coef_x_6  \
pow_1   3.7         2    -0.65      NaN      NaN      NaN      NaN      NaN   
pow_2   3.7       1.9    -0.54   -0.017      NaN      NaN      NaN      NaN   
pow_3   1.1      -1.4      3.4     -1.4     0.15      NaN      NaN      NaN   
pow_4   1.1      -1.1      2.9     -1.1    0.087   0.0051      NaN      NaN   
pow_5     1       0.7    -0.86      1.8    -0.97     0.18   -0.012      NaN   
pow_6     1      -6.1       16      -15      7.4       -2     0.28   -0.016   
pow_7  0.98       -19       54      -61       36      -13      2.5    -0.26   
pow_8  0.94       -66  2.1e+02 -2.9e+02  2.1e+02      -93       26     -4.2   
pow_9  0.94       -70  2.3e+02 -3.1e+02  2.4e+02 -1.1e+02       31     -5.5   
pow_10 0.88  -4.6e+02  1.9e+03 -3.4e+03  3.5e+03 -2.3e+03  9.9e+02 -2.9e+02   
pow_11 0.88  -5.4e+02  2.3e+03 -4.2e+03  4.4e+03   -3e+03  1.4e+03 -4.3e+02   
pow_12 0.88  -9.9e+02  4.6e+03 -9.4e+03  1.1e+04 -9.2e+03  5.1e+03 -2.1e+03   
pow_13 0.88  -1.4e+03  6.8e+03 -1.5e+04    2e+04 -1.7e+04  1.1e+04 -4.8e+03   
pow_14 0.87   2.5e+03 -1.7e+04  4.9e+04 -8.3e+04  9.5e+04 -7.6e+04  4.4e+04   
pow_15 0.87   1.8e+03 -1.2e+04  3.5e+04 -5.9e+04  6.6e+04 -5.1e+04  2.8e+04   

       coef_x_7 coef_x_8 coef_x_9 coef_x_10 coef_x_11 coef_x_12 coef_x_13  \
pow_1       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_2       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_3       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_4       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_5       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_6       NaN      NaN      NaN       NaN       NaN       NaN       NaN   
pow_7     0.011      NaN      NaN       NaN       NaN       NaN       NaN   
pow_8      0.39   -0.015      NaN       NaN       NaN       NaN       NaN   
pow_9      0.57    -0.03  0.00054       NaN       NaN       NaN       NaN   
pow_10       57     -7.3     0.53    -0.017       NaN       NaN       NaN   
pow_11       93      -13      1.2    -0.062    0.0013       NaN       NaN   
pow_12  5.9e+02 -1.2e+02       18      -1.7       0.1   -0.0027       NaN   
pow_13  1.6e+03 -3.9e+02       70      -8.9      0.75    -0.038   0.00088   
pow_14 -1.9e+04  6.1e+03 -1.5e+03   2.6e+02       -32       2.7     -0.14   
pow_15 -1.1e+04  3.4e+03 -6.9e+02        94      -6.6     -0.17     0.083   

       coef_x_14 coef_x_15  
pow_1        NaN       NaN  
pow_2        NaN       NaN  
pow_3        NaN       NaN  
pow_4        NaN       NaN  
pow_5        NaN       NaN  
pow_6        NaN       NaN  
pow_7        NaN       NaN  
pow_8        NaN       NaN  
pow_9        NaN       NaN  
pow_10       NaN       NaN  
pow_11       NaN       NaN  
pow_12       NaN       NaN  
pow_13       NaN       NaN  
pow_14    0.0032       NaN  
pow_15   -0.0071   0.00022